In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import colors
import json

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2024/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2024/sample_submission.json
/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json


In [6]:
def load_json_file(path):
    with open(path) as file:
        return json.load(file)

eval_challenges = load_json_file("/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json")
eval_solutions = load_json_file("/kaggle/input/arc-prize-2024/arc-agi_evaluation_solutions.json")
test_challenges = load_json_file("/kaggle/input/arc-prize-2024/arc-agi_test_challenges.json")
train_challenges = load_json_file("/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json")
train_solutions = load_json_file("/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json")

In [7]:
cmap = colors.ListedColormap(['black', 'blue', 'red', 'green', 'yellow', 'gray', 'magenta', 'orange', 'cyan', 'brown'])
norm = colors.Normalize(vmin=0, vmax=9)

def plot_task(task, task_solution=None):
    n_train_pairs = len(task['train'])
    n_test_pairs = len(task['test'])
    
    fig, axs = plt.subplots(n_train_pairs + n_test_pairs, 2, figsize=(5, 5 * (n_train_pairs + n_test_pairs)))
    fig.suptitle('Task')
    
    for i in range(n_train_pairs):
        axs[i, 0].imshow(task['train'][i]['input'], cmap=cmap, norm=norm)
        axs[i, 0].set_title('Train Input')
        axs[i, 0].axis('off')
        
        axs[i, 1].imshow(task['train'][i]['output'], cmap=cmap, norm=norm)
        axs[i, 1].set_title('Train Output')
        axs[i, 1].axis('off')
            
    for i in range(n_test_pairs):
        axs[n_train_pairs + i, 0].imshow(task['test'][i]['input'], cmap=cmap, norm=norm)
        axs[n_train_pairs + i, 0].set_title('Test Input')
        axs[n_train_pairs + i, 0].axis('off')
        
        if 'output' in task['test'][i]:
            axs[n_train_pairs + i, 1].imshow(task['test'][i]['output'], cmap=cmap, norm=norm)
            axs[n_train_pairs + i, 1].set_title('Test Output (Ground Truth)')
        elif task_solution:
            axs[n_train_pairs + i, 1].imshow(task_solution, cmap=cmap, norm=norm)
            axs[n_train_pairs + i, 1].set_title('Test Output (Predicted)')
            
        axs[n_train_pairs + i, 1].axis('off')
             
    plt.show()

In [9]:
#Remove the condition if you want to execute this blcok.
if 1 == 0:
    for task_id, task in train_challenges.items():
        solutions = train_solutions.get(task_id, [])
        if solutions:
            for solution in solutions:
                plt.figure()
                plot_task(task, solution)

In [ ]:
#Remove the condition if you want to execute this blcok.
if 1 == 0:
    example_task = train_challenges['007bbfb7']
    example_solution = train_solutions['007bbfb7'][0]
    print(example_task, example_solution)
    plot_task(example_task, example_solution)

In [ ]:
# Custom dataset class
class ReasoningDataset(Dataset):
    def __init__(self, data, train=True):
        self.data = data['train'] if train else data['test']
        self.train = train

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_image = torch.tensor(self.data[idx]['input'], dtype=torch.float32).unsqueeze(0) / 7.0
        if self.train:
            output_image = torch.tensor(self.data[idx]['output'], dtype=torch.float32).unsqueeze(0) / 7.0
            return input_image, output_image
        else:
            return input_image

In [ ]:
class CNNReasoningModel(nn.Module):
    def __init__(self):
        super(CNNReasoningModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.upsample = nn.Upsample(scale_factor=3, mode='bilinear', align_corners=True)
        self.conv4 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(32, 16, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(16, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = self.upsample(x)
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = torch.sigmoid(self.conv6(x))
        return x

In [ ]:
# Hyperparameters
batch_size = 2
learning_rate = 0.001
num_epochs = 10


# Create datasets and dataloaders
train_dataset = ReasoningDataset(data, train=True)
test_dataset = ReasoningDataset(data, train=False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Initialize the model, loss function, and optimizer
model = CNNReasoningModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Training Complete")

# Testing loop (predict outputs for test inputs)
model.eval()
test_results = []
with torch.no_grad():
    for inputs in test_loader:
        outputs = model(inputs)
        test_results.append(outputs.squeeze(0).squeeze(0).numpy())

print("Testing Complete")